In [1]:
import os

import gurobipy as gp
from gurobipy import GRB
import numpy as np

# Defining functions and formulations

In [2]:
# set index r for 2nd and 3rd formulations
# r = np.random.randint(n)
r2 = 5 
r3 = 5

In [3]:
def read_data(dir='data'):
    data = dict()
    files = os.listdir(dir)  # Lists all files and folders
    # Filter only files (exclude directories)
    files = [f for f in files if os.path.isfile(os.path.join(dir, f))]
    for f in files:
        name = f.split('.')
        if name[1] == 'txt':
            name = name[0]
            # reading the file
            with open(os.path.join(dir, f), "r") as ff:
                lines = ff.readlines()
            
            # Extract number of nodes
            num_nodes = int(lines[0].strip())
            # Load distance matrix
            distance_matrix = np.loadtxt(lines[1:], dtype=int)
            data[name] = distance_matrix
    return data

In [4]:
def print_results(model:gp.Model):
    if model.status == GRB.OPTIMAL:
        logFile = model.getParamInfo('logFile')[2]
        with open(logFile, 'a') as log:
            print("Optimal Solution:")
            log.write("\n \t Optimal Solution:\n")
            for v in model.getVars():
                if v.X != 0:  # 1 for x and f values and integer numbers for u values
                    print(f'{v.VarName}: {v.X}')
                    log.write(f'\t{v.VarName}: {v.X} \n')
            print(f'objective value: {model.Objval}')
            log.write(f'\tobjective value: {model.Objval}')
    else:
        print("No optimal solution found.")


In [5]:
def get_f0(data:np.array):
    n = data.shape[0]
    m = gp.Model('f0')
    
    # add variables
    x = m.addVars(n, n, vtype=GRB.BINARY, name='x')  # 1e
    
    m.addConstrs((gp.quicksum(x[i,j] for j in range(n))==1 for i in range(n)),
                 name='1b') 
    m.addConstrs((gp.quicksum(x[i,j] for i in range(n))==1 for j in range(n)),
                 name='1c')
    m.addConstrs((x[i, i] == 0 for i in range(n)),
                 name='1d')

    m.setObjective(gp.quicksum(data[i,j]*x[i,j] 
                               for i in range(n) 
                               for j in range(n)),
                   GRB.MINIMIZE)
    return m

In [6]:
def get_f2(data:np.array):
    n = data.shape[0]
    # similar to f0
    m = gp.Model('f2')
    x = m.addVars(n, n, vtype=GRB.BINARY, name='x')  # 1e
    m.addConstrs((gp.quicksum(x[i,j] for j in range(n))==1 for i in range(n)),
                 name='1b') 
    m.addConstrs((gp.quicksum(x[i,j] for i in range(n))==1 for j in range(n)),
                 name='1c')
    m.addConstrs((x[i, i] == 0 for i in range(n)),
                 name='1d')
    # new constrains for f2
    u = m.addVars(n, vtype=GRB.INTEGER, lb=1, name='u')
    r = r2
    
    m.addConstrs((u[i]-u[j]+1<=n*(1-x[i,j]) for i in range(n) for j in range(n) if i != r and j != r), name='3a')
    m.addConstr(u[r] == 1, name='3b')
    m.addConstrs((u[i]<=n for i in range(n) if i != r), name='3c1')
    m.addConstrs((u[i]>=2 for i in range(n) if i != r), name='3c2')  # added lower bound when defining 'u', do I need this now?
    
    m.setObjective(gp.quicksum(data[i,j]*x[i,j] 
                               for i in range(n) 
                               for j in range(n)),
                   GRB.MINIMIZE)
    return m


def run_f2(logfileName, params, datasets):
    for name,data in datasets.items():
        with gp.Env() as env:       
            with gp.Model(env=env) as model2:
                try:
                    m2 = get_f2(data)
                    m2.setParam("LogFile", f'logs/{logfileName}_{name}.log')
                    for key, value in params.items():
                        m2.setParam(key, value)
                    m2.optimize()
                    print_results(m2)
                except gp.GurobiError as e:
                    print('Error code ' + str(e.errno) + ': ' + str(e))
                except AttributeError:
                    print('Encountered an attribute error')

In [7]:
def get_f3(data:np.array):
    n = data.shape[0]
    # similar to f0
    m = gp.Model('f3')
    x = m.addVars(n, n, vtype=GRB.BINARY, name='x')  # 1e
    m.addConstrs((gp.quicksum(x[i,j] for j in range(n))==1 for i in range(n)),
                 name='1b') 
    m.addConstrs((gp.quicksum(x[i,j] for i in range(n))==1 for j in range(n)),
                 name='1c')
    m.addConstrs((x[i, i] == 0 for i in range(n)),
                 name='1d')
    # f3 constraints
    f = m.addVars(n, n, n, vtype=GRB.CONTINUOUS, lb=0, ub=1, name='f')
    r = r3
    
    m.addConstrs((gp.quicksum(f[v,r,j] for j in range(n) if j != r) - 
                  gp.quicksum(f[v,j,r] for j in range(n) if j != r) == 1  
                  for v in range(n) 
                  if v != r),
                 name='4a')
    
    m.addConstrs((gp.quicksum(f[v,i,j] for j in range(n) if j != i) - 
                  gp.quicksum(f[v,j,i] for j in range(n) if j != i) == 0  
                  for v in range(n) for i in range(n) 
                  if v != r and i != r and i != v), 
                 name='4b')
    
    m.addConstrs((f[v,i,j] <= x[i,j]   
                  for v in range(n) for i in range(n) for j in range(n) 
                  if v != r), 
                 name='4c')
    
    m.setObjective(gp.quicksum(data[i,j]*x[i,j] for i in range(n) for j in range(n)),
                   GRB.MINIMIZE)
    return m


def run_f3(logfileName, params, datasets):
    for name,data in datasets.items():
        with gp.Env() as env:       
            with gp.Model(env=env) as model3:
                try:
                    m3 = get_f3(data)
                    m3.setParam("LogFile", f'logs/{logfileName}_{name}.log')
                    
                    for key, value in params.items():
                        m3.setParam(key, value)
                    m3.optimize()
                    print_results(m3)
                except gp.GurobiError as e:
                    print('Error code ' + str(e.errno) + ': ' + str(e))
                except AttributeError:
                    print('Encountered an attribute error')

# Testing formulations



In [8]:
# reading datasets first
datasets = read_data('data')

### Test formulation 0 on four instances

In [9]:
for name, data in datasets.items():
    with gp.Env() as env:
    # with gp.Env(f'logs/f0_{name}.log') as env:
        with gp.Model(env=env) as model0:
            try:
                m0 = get_f0(data)
                m0.setParam("LogFile", f'logs/f0_{name}.log')
                m0.optimize()
                print_results(m0)
            except gp.GurobiError as e:
                print('Error code ' + str(e.errno) + ': ' + str(e))
            except AttributeError:
                print('Encountered an attribute error')

Set parameter Username
Set parameter LicenseID to value 2634667
Academic license - for non-commercial use only - expires 2026-03-10
Set parameter Username
Set parameter LicenseID to value 2634667
Academic license - for non-commercial use only - expires 2026-03-10
Set parameter LogFile to value "logs/f0_bays29.log"
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11+.0 (26100.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-12700, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 87 rows, 841 columns and 1711 nonzeros
Model fingerprint: 0x4cfafabf
Variable types: 0 continuous, 841 integer (841 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+01, 5e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 6280.0000000
Presolve removed 29 rows and 29 columns
Presolve time: 0.00s
Presolved: 58 rows, 812 

Thread count was 20 (of 20 available processors)

Solution count 2: 1089 8467 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.089000000000e+03, best bound 1.089000000000e+03, gap 0.0000%
Optimal Solution:
x[0,1]: 1.0
x[1,9]: 1.0
x[2,3]: 1.0
x[3,2]: 1.0
x[4,13]: 1.0
x[5,6]: 1.0
x[6,5]: 1.0
x[7,8]: 1.0
x[8,7]: 1.0
x[9,0]: 1.0
x[10,11]: 1.0
x[11,12]: 1.0
x[12,10]: 1.0
x[13,4]: 1.0
x[14,15]: 1.0
x[15,14]: 1.0
x[16,17]: 1.0
x[17,16]: 1.0
x[18,19]: 1.0
x[19,18]: 1.0
x[20,21]: 1.0
x[21,20]: 1.0
x[22,23]: 1.0
x[23,22]: 1.0
x[24,25]: 1.0
x[25,24]: 1.0
x[26,34]: 1.0
x[27,28]: 1.0
x[28,36]: 1.0
x[29,30]: 1.0
x[30,29]: 1.0
x[31,40]: 1.0
x[32,41]: 1.0
x[33,42]: 1.0
x[34,26]: 1.0
x[35,43]: 1.0
x[36,27]: 1.0
x[37,47]: 1.0
x[38,39]: 1.0
x[39,38]: 1.0
x[40,31]: 1.0
x[41,32]: 1.0
x[42,33]: 1.0
x[43,35]: 1.0
x[44,53]: 1.0
x[45,46]: 1.0
x[46,45]: 1.0
x[47,37]: 1.0
x[48,59]: 1.0
x[49,48]: 1.0
x[50,49]: 1.0
x[51,50]: 1.0
x[52,51]: 1.0
x[53,44]: 1.0
x[54,63]: 1.0
x[55,56]: 1.0
x[56,55]: 1.0
x[

### Test formulation 2 and 3 on four instances



#### a) with default settings: 

In [10]:
# FORMULATION 2
params = {'MIPGap':0, 'TimeLimit':3600}
run_f2('3a_f2', params, datasets)

Set parameter Username
Set parameter LicenseID to value 2634667
Academic license - for non-commercial use only - expires 2026-03-10
Set parameter LogFile to value "logs/3a_f2_bays29.log"
Set parameter MIPGap to value 0
Set parameter TimeLimit to value 3600
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11+.0 (26100.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-12700, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 928 rows, 870 columns and 4064 nonzeros
Model fingerprint: 0xe20bd28c
Variable types: 0 continuous, 870 integer (841 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [3e+01, 5e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+01]
Presolve removed 114 rows and 30 columns
Presolve time: 0.00s
Presolved: 814 rows, 840 columns, 3892 nonzeros
Variable types: 0 continuous, 840 integer (812 binary)
Found heuristic solution: o

     0     0  637.50000    0   97  718.00000  637.50000  11.2%     -    0s
     0     0  637.50000    0   98  718.00000  637.50000  11.2%     -    0s
     0     0  638.00000    0  106  718.00000  638.00000  11.1%     -    0s
     0     0  638.00000    0  107  718.00000  638.00000  11.1%     -    0s
     0     0  638.00000    0   99  718.00000  638.00000  11.1%     -    0s
H    0     0                     713.0000000  638.00000  10.5%     -    0s
     0     0  638.00000    0  103  713.00000  638.00000  10.5%     -    0s
     0     0  638.00000    0   93  713.00000  638.00000  10.5%     -    0s
     0     2  638.00000    0   85  713.00000  638.00000  10.5%     -    0s
H   28    40                     710.0000000  638.20306  10.1%  30.2    0s
H   87   100                     704.0000000  638.65094  9.28%  14.7    0s
* 3381  1709              42     701.0000000  681.66641  2.76%  12.0    1s
* 3675  1533              36     700.0000000  682.52317  2.50%  12.3    2s
H 3739  1434             

H44332 15133                    109404.00000 106547.108  2.61%  25.1   81s
 47409 16508 106548.405   76  133 109404.000 106547.108  2.61%  25.3   85s
 52030 18073 109214.312  102  166 109404.000 106547.108  2.61%  25.3   90s
 56005 19626 108238.011  106   60 109404.000 106547.108  2.61%  25.3   96s
 59071 20953 107685.500   79   65 109404.000 106555.955  2.60%  25.3  100s
 64158 22459 108172.111  169  107 109404.000 106584.250  2.58%  25.5  105s
*66968 22103              92    109386.00000 106596.667  2.55%  25.7  108s
 69042 22672 108414.200  100   33 109386.000 106604.809  2.54%  25.8  111s
 73922 24353 106876.345   82  191 109386.000 106644.614  2.51%  26.1  116s
*75660 18176              94    108540.00000 106654.338  1.74%  26.0  117s
 78176 18354     cutoff  102      108540.000 106682.344  1.71%  26.3  120s
 82687 19535 107444.775   77  149 108540.000 106739.525  1.66%  26.8  125s
H87389 18631                    108343.00000 106798.745  1.43%  27.3  129s
 87392 18624 108191.478  

H  181   193                    1233.0000000 1203.00000  2.43%  29.8    5s
H  182   193                    1216.0000000 1203.00000  1.07%  29.7    5s
H  186   193                    1214.0000000 1203.00000  0.91%  29.2    5s
H  187   193                    1212.0000000 1203.00000  0.74%  29.0    5s
  1906   652 infeasible   15      1212.00000 1203.32895  0.72%  23.8   10s
* 4072   587              16    1211.0000000 1205.78912  0.43%  25.6   13s
  4818   244     cutoff   12      1211.00000 1207.02806  0.33%  26.3   15s

Cutting planes:
  Learned: 21
  Gomory: 21
  Lift-and-project: 2
  Cover: 10
  Implied bound: 156
  MIR: 157
  StrongCG: 6
  Inf proof: 48
  Zero half: 2
  Mod-K: 1
  RLT: 21
  Relax-and-lift: 89
  BQP: 11
  PSD: 13

Explored 5293 nodes (138783 simplex iterations) in 15.41 seconds (11.25 work units)
Thread count was 20 (of 20 available processors)

Solution count 10: 1211 1212 1214 ... 6074

Optimal solution found (tolerance 0.00e+00)
Best objective 1.211000000000e+03, 

In [ ]:
# FORMULATION 3
params = {'MIPGap':0, 'TimeLimit':3600}
run_f3('3a_f3', params, datasets)

Set parameter Username
Set parameter LicenseID to value 2634667
Academic license - for non-commercial use only - expires 2026-03-10
Set parameter LogFile to value "logs/3a_f3_bays29.log"
Set parameter MIPGap to value 0
Set parameter TimeLimit to value 3600
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11+.0 (26100.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-12700, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 24419 rows, 25230 columns and 92711 nonzeros
Model fingerprint: 0x1712e753
Variable types: 24389 continuous, 841 integer (841 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+01, 5e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 1625 rows and 2466 columns
Presolve time: 0.24s
Presolved: 22794 rows, 22764 columns, 88648 nonzeros
Variable types: 21952 continuous, 812 integer (812 binary)

Root 

Set parameter LicenseID to value 2634667
Academic license - for non-commercial use only - expires 2026-03-10
Set parameter LogFile to value "logs/3a_f3_dantzig42.log"
Set parameter MIPGap to value 0
Set parameter TimeLimit to value 3600
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11+.0 (26100.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-12700, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 74131 rows, 75852 columns and 286060 nonzeros
Model fingerprint: 0x2b3b1d3e
Variable types: 74088 continuous, 1764 integer (1764 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+00, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 3445 rows and 5209 columns
Presolve time: 1.01s
Presolved: 70686 rows, 70643 columns, 277447 nonzeros
Variable types: 68921 continuous, 1722 integer (1722 binary)
Deterministic concur

Set parameter LicenseID to value 2634667
Academic license - for non-commercial use only - expires 2026-03-10
Set parameter LogFile to value "logs/3a_f3_pr76.log"
Set parameter MIPGap to value 0
Set parameter TimeLimit to value 3600
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11+.0 (26100.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-12700, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 439053 rows, 444752 columns and 1721778 nonzeros
Model fingerprint: 0xe3d5fa2a
Variable types: 438976 continuous, 5776 integer (5776 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+02, 2e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 5776 rows and 11552 columns (presolve time = 5s) ...
Presolve removed 5776 rows and 11552 columns
Presolve time: 9.29s
Presolved: 433277 rows, 433200 columns, 1710150 nonzeros
Variable

  2819   831 107606.000   18   46 108159.000 106805.000  1.25%  1912  809s
  3005   890     cutoff   12      108159.000 106837.167  1.22%  1876  823s
  3222   915 107268.667   13   81 108159.000 106875.417  1.19%  1836  838s
  3413   926     cutoff   15      108159.000 106891.000  1.17%  1817  861s
  3443   939 infeasible   16      108159.000 106912.000  1.15%  1815  875s
  3685   963 108004.500   13   61 108159.000 106925.500  1.14%  1774  889s
  3914   986     cutoff    9      108159.000 107016.500  1.06%  1743  903s
  4173  1018 107780.000   14   44 108159.000 107057.000  1.02%  1707  917s
  4430  1056     cutoff   20      108159.000 107084.000  0.99%  1678  932s
  4729  1083 107469.500   18   31 108159.000 107107.500  0.97%  1640  952s
  5060  1090     cutoff   16      108159.000 107141.000  0.94%  1599  970s
  5413  1107 108071.000   19   36 108159.000 107184.500  0.90%  1560  987s
  5734  1085 107991.000   19   32 108159.000 107200.500  0.89%  1537 1007s
  6130  1067 107749.500  

f[13,40,59]: 1.0
f[13,41,42]: 1.0
f[13,42,27]: 1.0
f[13,43,47]: 1.0
f[13,44,43]: 1.0
f[13,45,44]: 1.0
f[13,46,68]: 1.0
f[13,47,46]: 1.0
f[13,48,50]: 1.0
f[13,49,48]: 1.0
f[13,50,65]: 1.0
f[13,51,52]: 1.0
f[13,52,53]: 1.0
f[13,53,41]: 1.0
f[13,54,51]: 1.0
f[13,55,54]: 1.0
f[13,56,55]: 1.0
f[13,57,56]: 1.0
f[13,58,57]: 1.0
f[13,59,58]: 1.0
f[13,60,40]: 1.0
f[13,61,60]: 1.0
f[13,62,61]: 1.0
f[13,63,62]: 1.0
f[13,64,70]: 1.0
f[13,65,64]: 1.0
f[13,66,49]: 1.0
f[13,67,69]: 1.0
f[13,68,67]: 1.0
f[13,69,66]: 1.0
f[13,70,71]: 1.0
f[13,71,72]: 1.0
f[13,72,63]: 1.0
f[13,73,13]: 1.0
f[13,74,75]: 1.0
f[13,75,0]: 1.0
f[14,0,22]: 1.0
f[14,1,74]: 1.0
f[14,2,1]: 1.0
f[14,3,2]: 1.0
f[14,4,3]: 1.0
f[14,5,4]: 1.0
f[14,15,14]: 1.0
f[14,16,15]: 1.0
f[14,17,16]: 1.0
f[14,18,30]: 1.0
f[14,19,18]: 1.0
f[14,20,24]: 1.0
f[14,21,20]: 1.0
f[14,22,21]: 1.0
f[14,23,45]: 1.0
f[14,24,23]: 1.0
f[14,25,19]: 1.0
f[14,26,25]: 1.0
f[14,27,26]: 1.0
f[14,28,31]: 1.0
f[14,29,28]: 1.0
f[14,30,29]: 1.0
f[14,31,32]: 1.0
f[14,32,

f[36,44,43]: 1.0
f[36,45,44]: 1.0
f[36,46,68]: 1.0
f[36,47,46]: 1.0
f[36,48,50]: 1.0
f[36,49,48]: 1.0
f[36,50,65]: 1.0
f[36,51,52]: 1.0
f[36,52,53]: 1.0
f[36,53,41]: 1.0
f[36,54,51]: 1.0
f[36,55,54]: 1.0
f[36,56,55]: 1.0
f[36,57,56]: 1.0
f[36,58,57]: 1.0
f[36,59,58]: 1.0
f[36,60,40]: 1.0
f[36,61,60]: 1.0
f[36,62,61]: 1.0
f[36,63,62]: 1.0
f[36,64,70]: 1.0
f[36,65,64]: 1.0
f[36,66,49]: 1.0
f[36,67,69]: 1.0
f[36,68,67]: 1.0
f[36,69,66]: 1.0
f[36,70,71]: 1.0
f[36,71,72]: 1.0
f[36,72,63]: 1.0
f[36,74,75]: 1.0
f[36,75,0]: 1.0
f[37,0,22]: 1.0
f[37,1,74]: 1.0
f[37,2,1]: 1.0
f[37,3,2]: 1.0
f[37,4,3]: 1.0
f[37,5,4]: 1.0
f[37,18,30]: 1.0
f[37,19,18]: 1.0
f[37,20,24]: 1.0
f[37,21,20]: 1.0
f[37,22,21]: 1.0
f[37,23,45]: 1.0
f[37,24,23]: 1.0
f[37,25,19]: 1.0
f[37,26,25]: 1.0
f[37,27,26]: 1.0
f[37,28,31]: 1.0
f[37,29,28]: 1.0
f[37,30,29]: 1.0
f[37,31,32]: 1.0
f[37,32,34]: 1.0
f[37,33,39]: 1.0
f[37,34,33]: 1.0
f[37,38,37]: 1.0
f[37,39,38]: 1.0
f[37,40,59]: 1.0
f[37,41,42]: 1.0
f[37,42,27]: 1.0
f[37,43,

f[53,64,70]: 1.0
f[53,65,64]: 1.0
f[53,66,49]: 1.0
f[53,67,69]: 1.0
f[53,68,67]: 1.0
f[53,69,66]: 1.0
f[53,70,71]: 1.0
f[53,71,72]: 1.0
f[53,72,63]: 1.0
f[53,74,75]: 1.0
f[53,75,0]: 1.0
f[54,0,22]: 1.0
f[54,1,74]: 1.0
f[54,2,1]: 1.0
f[54,3,2]: 1.0
f[54,4,3]: 1.0
f[54,5,4]: 1.0
f[54,20,24]: 1.0
f[54,21,20]: 1.0
f[54,22,21]: 1.0
f[54,23,45]: 1.0
f[54,24,23]: 1.0
f[54,40,59]: 1.0
f[54,43,47]: 1.0
f[54,44,43]: 1.0
f[54,45,44]: 1.0
f[54,46,68]: 1.0
f[54,47,46]: 1.0
f[54,48,50]: 1.0
f[54,49,48]: 1.0
f[54,50,65]: 1.0
f[54,55,54]: 1.0
f[54,56,55]: 1.0
f[54,57,56]: 1.0
f[54,58,57]: 1.0
f[54,59,58]: 1.0
f[54,60,40]: 1.0
f[54,61,60]: 1.0
f[54,62,61]: 1.0
f[54,63,62]: 1.0
f[54,64,70]: 1.0
f[54,65,64]: 1.0
f[54,66,49]: 1.0
f[54,67,69]: 1.0
f[54,68,67]: 1.0
f[54,69,66]: 1.0
f[54,70,71]: 1.0
f[54,71,72]: 1.0
f[54,72,63]: 1.0
f[54,74,75]: 1.0
f[54,75,0]: 1.0
f[55,0,22]: 1.0
f[55,1,74]: 1.0
f[55,2,1]: 1.0
f[55,3,2]: 1.0
f[55,4,3]: 1.0
f[55,5,4]: 1.0
f[55,20,24]: 1.0
f[55,21,20]: 1.0
f[55,22,21]: 1.0
f[

f[70,64,70]: 1.0
f[70,65,64]: 1.0
f[70,66,49]: 1.0
f[70,67,69]: 1.0
f[70,68,67]: 1.0
f[70,69,66]: 1.0
f[70,74,75]: 1.0
f[70,75,0]: 1.0
f[71,0,22]: 1.0
f[71,1,74]: 1.0
f[71,2,1]: 1.0
f[71,3,2]: 1.0
f[71,4,3]: 1.0
f[71,5,4]: 1.0
f[71,20,24]: 1.0
f[71,21,20]: 1.0
f[71,22,21]: 1.0
f[71,23,45]: 1.0
f[71,24,23]: 1.0
f[71,43,47]: 1.0
f[71,44,43]: 1.0
f[71,45,44]: 1.0
f[71,46,68]: 1.0
f[71,47,46]: 1.0
f[71,48,50]: 1.0
f[71,49,48]: 1.0
f[71,50,65]: 1.0
f[71,64,70]: 1.0
f[71,65,64]: 1.0
f[71,66,49]: 1.0
f[71,67,69]: 1.0
f[71,68,67]: 1.0
f[71,69,66]: 1.0
f[71,70,71]: 1.0
f[71,74,75]: 1.0
f[71,75,0]: 1.0
f[72,0,22]: 1.0
f[72,1,74]: 1.0
f[72,2,1]: 1.0
f[72,3,2]: 1.0
f[72,4,3]: 1.0
f[72,5,4]: 1.0
f[72,20,24]: 1.0
f[72,21,20]: 1.0
f[72,22,21]: 1.0
f[72,23,45]: 1.0
f[72,24,23]: 1.0
f[72,43,47]: 1.0
f[72,44,43]: 1.0
f[72,45,44]: 1.0
f[72,46,68]: 1.0
f[72,47,46]: 1.0
f[72,48,50]: 1.0
f[72,49,48]: 1.0
f[72,50,65]: 1.0
f[72,64,70]: 1.0
f[72,65,64]: 1.0
f[72,66,49]: 1.0
f[72,67,69]: 1.0
f[72,68,67]: 1.0
f[

#### b) with `presolve` truned off: 

In [ ]:
# FORMULATION 2
params = {'MIPGap':0, 'TimeLimit':3600, 'Presolve':0}
run_f2('3b_f2', params, datasets)

In [ ]:
# FORMULATION 3
params = {'MIPGap':0, 'TimeLimit':3600, 'Presolve':0}
run_f3('3b_f3', params, datasets)

#### c) with `cuts` turned off: 

In [ ]:
# FORMULATION 2
params = {'MIPGap':0, 'TimeLimit':3600, 'Cuts':0}
run_f2('3c_f2', params, datasets)

In [ ]:
# FORMULATION 3
params = {'MIPGap':0, 'TimeLimit':3600, 'Cuts':0}
run_f3('3c_f3', params, datasets)

#### d) with both `presolve` and `cuts` turned off

In [ ]:
# FORMULATION 2
params = {'MIPGap':0, 'TimeLimit':3600, 'Presolve':0, 'Cuts':0}
run_f2('3d_f2', params, datasets)

In [ ]:
# FORMULATION 3
params = {'MIPGap':0, 'TimeLimit':3600, 'Presolve':0, 'Cuts':0}
run_f3('3d_f3', params, datasets)